<a href="https://colab.research.google.com/github/codesanta/Protest-News/blob/master/protest_news_summ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This Jupyter Notebook Has the Code which Takes the Full Articles and Summarises them using PageRank algorithm and Compares the results with the reference to generate rouge scores for evaluations. 

In [ ]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import nltk
nltk.download('stopwords')
import pandas as pd
# data=pd.read_csv("traindata.csv")
# cont=data.content
# text=cont[0]
 
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    data=pd.read_csv("traindata.csv")
    cont=data.content
    text=cont[2]
    article = text.split(". ")
    sentences = []

    for sentence in article:
        #print(sentence)
        lel = sentence.replace("[^a-zA-Z]", " ").split(" ")
        # # lel.remove("")
        # lel.remove(" ")
        # lel.remove("  ")
        sentences.append(lel)
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)
    #print(sentences)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))

# let's begin
generate_summary( "file.txt", 2)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Summarize Text: 
 Residents of Javagal hobli in Arsikere taluk, here on Wednesday, staged a protest demanding fair compensation for their land being acquired for Yettinahole project, besides water for the villages under the project.
Hundreds of people from the hobli staged a dharna in front of the Deputy Commissioner's office. “We are demanding the same compensation for our land


In [ ]:
On 23 December, veteran artistes and poets including members of AASU assembled in Guwahati (Kamrup district, Assam) and held protest against the Citizenship (Amendment) Act, 2019. [size=no report]

In [ ]:
pip install rouge

In [ ]:
from rouge import Rouge 

hypothesis = "Residents of Javagal hobli in Arsikere taluk, here on Wednesday, staged a protest demanding fair compensation for their land being acquired for Yettinahole project, besides water for the villages under the project.Hundreds of people from the hobli staged a dharna in front of the Deputy Commissioner's office. “We are demanding the same compensation for our land"
reference = "On December 4, hundreds of residents staged a protest in front of the Deputy Commissioner's office in Hassan city (Hassan, Karnataka), demanding fair compensation for their land being acquired for Yettinahole project, besides water for the villages under the project."
rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)

In [ ]:
print(scores)

[{'rouge-1': {'f': 0.6391752528855352, 'p': 0.543859649122807, 'r': 0.775}, 'rouge-2': {'f': 0.5473684162127425, 'p': 0.4642857142857143, 'r': 0.6666666666666666}, 'rouge-l': {'f': 0.6666666617102223, 'p': 0.6097560975609756, 'r': 0.7352941176470589}}]
